In [1]:
import pandas as pd
import requests
from io import StringIO

apple_reviews = '' # get cdn url from the discord
google_reviews = ''

def read_url_csv(url):
    s = requests.get(url).text

    data = StringIO(s) 
    df = pd.read_csv(data)
    return df

apple_reviews = read_url_csv(apple_reviews)
google_reviews = read_url_csv(google_reviews)

In [2]:
import pandas as pd

apple_reviews.columns

# Extract 'content' column from google_reviews and rename it to 'review'
google_reviews_subset = google_reviews[['content']].rename(columns={'content': 'review'})

# Extract 'review' column from apple_reviews
apple_reviews_subset = apple_reviews[['review']]

# Concatenate the two dataframes
df = pd.concat([google_reviews_subset, apple_reviews_subset], ignore_index=True)


In [3]:
%pip install -U sentence_transformers
from sentence_transformers import SentenceTransformer
import pandas as pd
model = SentenceTransformer('all-mpnet-base-v2').to('cuda')
sentence_embeddings = model.encode(list(df['review']), show_progress_bar=True)

Note: you may need to restart the kernel to use updated packages.


Batches:   0%|          | 0/442 [00:00<?, ?it/s]

In [4]:
#%pip install MulticoreTSNE
from MulticoreTSNE import MulticoreTSNE as TSNE
import re 

tsne = TSNE(n_jobs=4, perplexity=50)
Y = tsne.fit_transform(sentence_embeddings)
df['reduced_dimension_embedding_1'] = Y[:,0]
df['reduced_dimension_embedding_2'] = Y[:,1]

In [ ]:
import altair as alt 
# plot a scatterplot using altair
alt.data_transformers.disable_max_rows()
chart = alt.Chart(df).mark_circle(size=20).encode(
  x=alt.X('reduced_dimension_embedding_1', axis=alt.Axis(title='', grid=False)),
  y=alt.Y('reduced_dimension_embedding_2', axis=alt.Axis(title='', grid=False)),
  tooltip=['review']
).properties(
    width=512,
    height=512
).interactive()
chart

In [8]:
chart.save('clustering_reviews.html')